# 券商分點

這個券商分點可以繞過「圖片認證碼」請大家再三保密使用XD
不然哪天就失效了，

抓取券商分點，每檔股票總共可以分成三個步驟：
1. 開啟驗證碼網頁，得到驗證碼
2. 利用驗證碼來得到券商分點
3. 整理券商分點資訊

In [1]:
# ---------------
# 偽裝成瀏覽器
# ---------------
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0.1) Gecko/2010010' \
    '1 Firefox/4.0.1',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':'en-us,en;q=0.5',
    'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.7'}

# 1. 開啟驗證碼網頁，得到驗證碼
這邊的驗證碼不是「圖片驗證碼喔！」而是 `viewstate` 跟 `eventvalidation`

In [2]:
# ---------------
# 開啟有驗證碼的網頁
# ---------------
import requests
rs = requests.session()

def get_verify_code(rs):
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsMenu.aspx', stream = True, verify = False, headers = headers, timeout=None )
    # ---------------
    # 從網頁中抓取一些關鍵字
    # ---------------
    import re
    
    # get view state
    viewstate = re.search( 'VIEWSTATE"\s+value=.*=', res.text )
    viewstate = viewstate.group()[18:]
    
    # get eventvalidation
    eventvalidation = re.search( 'EVENTVALIDATION"\s+value=.*\w', res.text )
    eventvalidation = eventvalidation.group( )[ 24: ]
    
    return viewstate, eventvalidation

viewstate, eventvalidation = get_verify_code(rs)

print('----')
print('viewstate', viewstate)
print('----')
print('eventvalidation',eventvalidation)
print('----')

----
viewstate /wEPDwUJOTAxMjkxMjg3D2QWAgIDD2QWBAIBD2QWAmYPZBYEZg9kFgJmDxAPFgYeB0NoZWNrZWRnHglGb3JlQ29sb3IKAB4EXyFTQgIEZGRkZAIBD2QWAgIBDw8WBB8BCgAfAgIEZGQCAw9kFgQCAw8PFgIeBFRleHRlZGQCCQ8PFgIeB1Zpc2libGVoZGQYAgUeX19Db250cm9sc1JlcXVpcmVQb3N0QmFja0tleV9fFgMFElJhZGlvQnV0dG9uX05vcm1hbAUQUmFkaW9CdXR0b25fRXhjZAUQUmFkaW9CdXR0b25fRXhjZAUPQ2FwdGNoYUNvbnRyb2wxDwUkNWNmODQzOGYtYTU2Ny00NzI1LTk2MGQtYmU3ZDgwZTA0MGVkZEHBZfHChQjOvwSIl4RVkmo=
----
eventvalidation /wEdAAYn8oS2my24F9fe7mvFP6cHfMmuxAJNAJcNkRRsVeJwwqKURZav/+YrVMqaWE2hvMxJwK4Ohf3nRgXHkFhQen1PRSciNdPYWmINCrip1wqw01PJCA0uL9aE2sjICZqv6GqT8tiOISoQ2Lix96ZtImF9
----


# 2. 透過驗證碼爬取資料並存檔

In [3]:
# ---------------
# 爬取資料
# ---------------

def get_data(rs, stock_id, viewstate, eventvalidation):
    import time
    payload = {
    '__EVENTTARGET':'',
    '__EVENTARGUMENT':'',
    '__LASTFOCUS':'',
    '__VIEWSTATE' : viewstate,                      #encode_viewstate[:-1],
    '__EVENTVALIDATION' : eventvalidation,          #encode_eventvalidation[:-1],
    'RadioButton_Normal' : 'RadioButton_Normal',
    'TextBox_Stkno' : stock_id,
    'CaptchaControl1 ' : 'Z67YB',
    'btnOK' : '%E6%9F%A5%E8%A9%A2',
    }

    rs.post( "http://bsr.twse.com.tw/bshtm/bsMenu.aspx", data=payload, headers=headers)#, verify = False, stream = True )
    time.sleep(1)
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsContent.aspx')#,verify = False ,stream = True, )
    return res

res = get_data(rs, '1101', viewstate, eventvalidation)
res.encoding='big5'
open('test.html', 'w', encoding='utf-8').write(res.text)

38076

# 3. 整理資料

In [4]:
import pandas as pd
from io import StringIO

def parse_data(text):
    lines = text.split('\n')
    lines = [l for l in lines if len(l.split(',')) == 11]
    df = pd.read_csv(StringIO('\n'.join(lines)))

    first_df = df[df.columns[:5]]
    second_df = df[df.columns[6:]]
    second_df.columns = second_df.columns.str.replace('.1', '')
    final_df = first_df.append(second_df).set_index('序號').sort_index().dropna()
    return final_df


df = parse_data(res.text)

# （每天來按一下）

假如無法下載，可以參考
http://finlabcourse.imotor.com/viewthread.php?tid=322&extra=page%3D1&page=3
來做一些更動，就可以順利執行了～

In [ ]:
import time
import requests
from io import StringIO
import pandas as pd
import datetime

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0.1) Gecko/2010010' \
    '1 Firefox/4.0.1',
    'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language':'en-us,en;q=0.5',
    'Accept-Charset':'ISO-8859-1,utf-8;q=0.7,*;q=0.7'}

def get_data(rs, stock_id, viewstate, eventvalidation):
    import time
    payload = {
    '__EVENTTARGET':'',
    '__EVENTARGUMENT':'',
    '__LASTFOCUS':'',
    '__VIEWSTATE' : viewstate,                      #encode_viewstate[:-1],
    '__EVENTVALIDATION' : eventvalidation,          #encode_eventvalidation[:-1],
    'RadioButton_Normal' : 'RadioButton_Normal',
    'TextBox_Stkno' : stock_id,
    'CaptchaControl1 ' : 'Z67YB',
    'btnOK' : '%E6%9F%A5%E8%A9%A2',
    }

    rs.post( "http://bsr.twse.com.tw/bshtm/bsMenu.aspx", data=payload, headers=headers)#, verify = False, stream = True )
    time.sleep(1)
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsContent.aspx')#,verify = False ,stream = True, )
    return res

def get_verify_code(rs):
    res = rs.get( 'http://bsr.twse.com.tw/bshtm/bsMenu.aspx', stream = True, verify = False, headers = headers, timeout=None )
    # ---------------
    # 從網頁中抓取一些關鍵字
    # ---------------
    import re
    
    # get view state
    viewstate = re.search( 'VIEWSTATE"\s+value=.*=', res.text )
    viewstate = viewstate.group()[18:]
    
    # get eventvalidation
    eventvalidation = re.search( 'EVENTVALIDATION"\s+value=.*\w', res.text )
    eventvalidation = eventvalidation.group( )[ 24: ]
    
    return viewstate, eventvalidation

def download_stock(stock_id, path):
    rs = requests.Session()
    #viewstate, eventvalidation = get_verify_code(rs)
    
    while True:
        try:
            viewstate, eventvalidation = get_verify_code(rs)
            break # 抓資料成功,進行下一步
        except Exception as e:
            print(e) # 印出無法連線原因
            print('無法拿到資料, 等 31 sec') # server好像會擋30sec
            time.sleep(31)
            continue # 抓資料失敗,重新抓資料
    
    time.sleep(2)
    res = get_data(rs, stock_id, viewstate, eventvalidation)
    res.encoding='big5'
    open(path, 'w', encoding='utf-8').writelines(res.text)
    
def stock_list():
    res = requests.get('http://dts.twse.com.tw/opendata/t187ap03_L.csv')
    res.encoding='big5'
    df = pd.read_csv(StringIO(res.text), index_col=['公司代號'])
    return df

import os

# build main dir
main_dir = os.path.join('data', 'broker_history')
if not os.path.isdir(main_dir):
    os.mkdir(main_dir)
    
# build date dir
now = datetime.datetime.now()
date_dir = os.path.join(main_dir, now.strftime('%Y%m%d'))
if not os.path.isdir(date_dir):
    os.mkdir(date_dir)

    
slist = stock_list()
for s in slist.index:
    stock_path = os.path.join(date_dir, str(s) + '.csv')
    print(stock_path)
    if not os.path.isfile(stock_path):
        print('downloading...')
        download_stock(str(s), stock_path)
        time.sleep(10)
    

data/broker_history/20181017/1101.csv
downloading...
data/broker_history/20181017/1102.csv
downloading...
data/broker_history/20181017/1103.csv
downloading...
data/broker_history/20181017/1104.csv
downloading...
data/broker_history/20181017/1108.csv
downloading...
data/broker_history/20181017/1109.csv
downloading...
data/broker_history/20181017/1110.csv
downloading...
data/broker_history/20181017/1201.csv
downloading...
data/broker_history/20181017/1203.csv
downloading...
data/broker_history/20181017/1210.csv
downloading...
data/broker_history/20181017/1213.csv
downloading...
data/broker_history/20181017/1215.csv
downloading...
data/broker_history/20181017/1216.csv
downloading...
data/broker_history/20181017/1217.csv
downloading...
data/broker_history/20181017/1218.csv
downloading...
data/broker_history/20181017/1219.csv
downloading...
data/broker_history/20181017/1220.csv
downloading...
data/broker_history/20181017/1225.csv
downloading...
data/broker_history/20181017/1227.csv
download

data/broker_history/20181017/1704.csv
downloading...
data/broker_history/20181017/1707.csv
downloading...
data/broker_history/20181017/1708.csv
downloading...
data/broker_history/20181017/1709.csv
downloading...
data/broker_history/20181017/1710.csv
downloading...
data/broker_history/20181017/1711.csv
downloading...
data/broker_history/20181017/1712.csv
downloading...
data/broker_history/20181017/1713.csv
downloading...
data/broker_history/20181017/1714.csv
downloading...
data/broker_history/20181017/1717.csv
downloading...
data/broker_history/20181017/1718.csv
downloading...
data/broker_history/20181017/1720.csv
downloading...
data/broker_history/20181017/1721.csv
downloading...
data/broker_history/20181017/1722.csv
downloading...
data/broker_history/20181017/1723.csv
downloading...
data/broker_history/20181017/1724.csv
downloading...
data/broker_history/20181017/1725.csv
downloading...
data/broker_history/20181017/1726.csv
downloading...
data/broker_history/20181017/1727.csv
download

# 券商總買賣

In [5]:
buy = df['買進股數'].astype(int).groupby(df['券商']).sum()
sell= df['賣出股數'].astype(int).groupby(df['券商']).sum()
total = buy - sell
total

券商
1020合　　庫      206000
1021合庫台中           0
1022合庫台南       -1000
1023合庫高雄      -50000
1024合庫嘉義          -4
1029合庫鳳山        6000
102C合庫自強        2000
1032土銀台南       15000
1035土銀新竹       12000
103F土銀南港        6000
1040臺    銀     48232
1041臺銀鳳山       -4000
1042臺銀臺南       11000
1043臺銀民權       13150
1045臺銀新竹      -95000
104A臺銀臺中        1900
104C臺銀高雄        3000
104D臺銀金山        3000
1115企銀太平         100
1116企銀屏東       51000
1117企銀竹北        -999
1118企銀豐原        1000
1160日    盛      6000
1163日盛台中       14000
1166日盛雙和           0
1167日盛嘉義        2000
1168日盛高雄        4000
116A日盛木柵       10000
116B日盛中壢        5000
116C日盛八德       -1000
               ...  
9A98永豐大園        1000
9A9B永豐中和       -1000
9A9D永豐忠孝       -2000
9A9E永豐板橋       -2000
9A9F永豐安和        2000
9A9G永豐天母        8000
9A9H永豐新莊        4000
9A9J永豐板新       -1000
9A9L永豐台中        3000
9A9N永豐桃盛       -3000
9A9Q永豐豐原        1000
9A9R永豐信義           0
9A9S永豐南京       -5000
9A9U永豐中正       -1000
9A9V永豐三民        2000
9A9W永豐市政       -6000
9A9X永豐竹科  

# 主力買賣超

In [6]:
# 買方top 15
total.nlargest(15)

券商
1650瑞銀        559000
9623富邦台北      522000
9A93永豐大稻      450000
1530德意志       437000
7000兆豐        409976
1020合　　庫      206000
592P元富同大      205000
1470台灣摩根      194000
9815元大大灣      150000
1360港麥格理      149000
9697富邦虎尾      132000
5922元富新店      130000
980h元大台北      100000
1570港商法興       93000
5050大    展     80000
dtype: int64

In [7]:
# 賣方 top 15
total.nsmallest(15)

券商
9100群益       -2258000
1520瑞士信貸      -814000
8960上海匯豐      -772000
1440美林        -631000
8440摩根大通      -325000
8150台    新    -309000
1480美商高盛      -195600
9692富邦嘉義       -96960
1045臺銀新竹       -95000
9800元大         -78906
9268凱基台北       -63000
9189群益桃園       -54000
5854統一城中       -52000
9229凱基中山       -51000
1023合庫高雄       -50000
dtype: int64

In [8]:
# 主力買賣超
total.nlargest(15).sum() + total.nsmallest(15).sum()

-2028490

# 買賣家數差

In [9]:
# 買賣家數差
(total > 0).sum() - (total < 0).sum()

275